# Filtracja bilateralna

## Konwolucja obrazu z filtrem o zadanych współczynnikach

Splot (konwolucję) obrazu wejściowego $I$ z filtrem $\psi$ dla ustalonego punktu obrazu $\mathbf{x}$ można przedstawić następująco:

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||)I(\mathbf{p})
\tag{1}
\end{equation}

gdzie:
- $\hat{I}$ jest obrazem wynikowym (przefiltrowanym),
- $W_N = \sum_y \psi(y)$ jest parametrem normalizującym współczynniki filtra $\psi$,
- $||\cdot||$ jest odległością między punktami obrazu $\mathbf{x}$ i $\mathbf{p}$ według ustalonej metryki (np. norma $L_2$). Uwaga, proszę pamiętać, że zarówno $\mathbf{x}$, jak i $\mathbf{p}$ to współrzędne przestrzenne,
- $\eta(\mathbf{x})$ jest otoczeniem punktu $\mathbf{x}$.

Funkcja $\psi$ decyduje o charakterze filtracji. Dla filtru Gaussowskiego:

\begin{equation}
\psi(y) = G_{\delta_s}(y)
\tag{2}
\end{equation}

gdzie: $G_{\delta_s}(y)$ jest funkcją Gaussa z parametrem skali $\delta_s$.

Opisaną powyżej filtrację realizowaliśmy w ramach ćwiczenia "Przetwarzanie wstępne. Filtracja kontekstowa."

## Filtracja bilateralna

Wadą klasycznego splotu jest brak adaptacji współczynników filtra do lokalnego otoczenia $\eta(\mathbf{x})$ filtrowanego punktu $\mathbf{x}$.
Oznacza to wykorzystanie tych samych współczynników filtra $\psi$ niezależnie od tego czy otoczenie jest względnie jednorodne lub zawiera krawędzie obiektów (w tym przypadku dochodzi do "rozmywania" krawędzi).
Filtracja bilateralna uwzględnia lokalne otoczenie filtrowanego punktu, w ten sposób, że parametry filtra zmieniają się w zależności od "wyglądu" otoczenia.


Współczynniki filtra obliczane są na podstawie odległości filtrowanego punktu $\mathbf{x}$ od każdego punktu otoczenia $\mathbf{p}$ w dziedzinie przestrzennej obrazu (tak jak przy typowym filtrze np. Gaussa) oraz odległości punktów w przeciwdziedzinie obrazu (np. na podstawie różnicy w jasności pikseli dla obrazu w odcieniach szarości):

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||) \gamma(|I(\mathbf{p}) - I(\mathbf{x})|) I(\mathbf{p})
\tag{3}
\end{equation}
gdzie:
- $W_N$ jest współczynnikiem normalizującym filtr,
- $\gamma$ jest funkcją odległości w przeciwdziedzinie obrazu, np. $\gamma(y)=\exp(-\frac{y^2}{2\delta_r^2})$
- parametr $\delta_r$ jest utożsamiany z poziomem szumu w obrazie i należy go dobrać w sposób empiryczny.

Proszę chwilę zastanowić się nad powyższym równaniem, w szczególności nad funkcją $\gamma$. Proszę wyznaczyć, jaka będzie wartość funkcji dla pikseli podobnych (różnica 0, 1, 2), a skrajnie różnych (255, 200).

##  Realizacja ćwiczenia

### Wczytanie danych

1. Wczytaj dane z pliku *MR_data.mat*. W tym celu wykorzystaj funkcję `loadmat` z pakietu scipy:
        from scipy.io import loadmat
        mat = loadmat('MR_data.mat')

2. Wczytany plik zawiera 5 obrazów: *I_noisefree*, *I_noisy1*, *I_noisy2*, *I_noisy3* oraz *I_noisy4*. Odczytać je można w następujący sposób:
        Input = mat['I_noisy1']

3. Wyświetl wybrany obraz z pliku *MR_data.mat*. Zagadka - co to za obrazowanie medyczne?

In [ ]:
from IPython.core.pylabtools import figsize
from matplotlib import pyplot as plt
import numpy as np
from scipy.io import loadmat
import math
import os


if not os.path.exists("MR_data.mat") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/MR_data.mat --no-check-certificate

#TODO Samodzielna

mat = loadmat('MR_data.mat')

Input1 = mat['I_noisy1']
Input2 = mat['I_noisy2']
Input3 = mat['I_noisy3']
Input4 = mat['I_noisy4']
Input_0 = mat['I_noisefree']

plt.imshow(Input3, 'gray')
plt.axis('off')
plt.show()

### "Klasyczna" konwolucja

1. Zdefiniuj parametry filtra Gaussowskiego: rozmiar okna i wariancję $\delta_S$.
2. Oblicz współczynniki filtra na podstawie zdefiniowanych parametrów (najprościej w ramach podwójnej pętli for).
2. Sprawdź ich poprawność i zwizualizuj filtr (tak jak w ćwiczeniu pt. "Przetwarzanie wstępne. Filtracja kontekstowa.").
3. Wykonaj kopię obrazu wejściowego: `IConv = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentem tej funkcji są aktualnie przetwarzane okno i współczynniki filtra.
7. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
8. Wyświetl wynik filtracji.
9. Porównaj wynik z obrazem oryginalnym.

In [ ]:
#TODO Samodzielna

def fgaussian(size, sigma):
  m = n = size
  h, k = m//2, n//2
  x, y = np.mgrid[-h:h+1, -k:k+1]
  g = np.exp(-(x**2 + y**2)/(2*sigma**2))
  return g


def mesh(fun, size):
  fig = plt.figure()
  ax = fig.gca(projection='3d')
  

  X = np.arange(-size//2, size//2, 1)
  Y = np.arange(-size//2, size//2, 1)
  X, Y = np.meshgrid(X, Y)
  Z = fun
  
  ax.plot_surface(X, Y, Z)
  
  plt.show()

maska = fgaussian(5,0.5)
mesh(maska,5)

In [ ]:
def new_pixel_value(window, size, sigma):
  mask = fgaussian(size, sigma)
  A,B = window.shape
  pixel = 0
  x = A//2
  y = B//2
  for i in range(A):
    for j in range(B):
      dst = np.sqrt((x - i)**2 + (y - j)**2)
      gaus = np.exp(-(dst**2)/(2*(sigma**2)))
      pixel = pixel + gaus*window[i,j]
  return pixel/mask.sum()

In [ ]:
def convol(image, size, sigma):
  IConv = image.copy()
  (X,Y) = IConv.shape
  var = size//2
  for i in range(0 + var, X - var):
    for j in range(0 + var, Y - var):
      window = IConv[i-var:i+var+1, j-var:j+var+1]
      new_pixel = new_pixel_value(window, size, sigma)
      IConv[i,j] = new_pixel
  return IConv

f, (ax1, ax2) = plt.subplots(1,2,figsize=(16,16))
ax1.imshow(Input1,'gray')
ax1.set_title("Onraz oryginalny")
ax1.axis('off')
ax2.imshow(convol(Input1,7, 0.7), 'gray')
ax2.set_title("Obraz w wyniku konwolucji")
ax2.axis('off')

W wyniku konwulcji obraz stał się odrobinę mniej przejrzysty. Krawędzie są bardziej rozmyte. Możemy zauważyć także występujące białe kropki. 

### Filtracja bilateralna

1. Zdefiniuj dodatkowy parametr: wariancję $\delta_R$.
3. Wykonaj kopię obrazu wejściowego: `IBilateral = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentami funkcji są aktualnie przetwarzane okno, współczynniki filtra gaussowskiego (takie same jak wcześniej) i wariancja $\delta_R$.
7. Oblicz odległość w przeciwdziedzinie (dla wartości pikseli).
8. Oblicz funkcję Gaussa dla obliczonych odległości z zadanym parametrem.
9. Wykonaj normalizację obliczonych współczynników.
10. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
11. Wyświetl wynik filtracji.
12. Porównaj wynik z obrazem oryginalnym.

In [ ]:
def new_pixel_value_2(window, sigma, delta_r):
  A,B = window.shape
  pixel = 0
  x = A//2
  y = B//2
  normalization = 0
  for i in range(A):
    for j in range(B):
      dst = np.sqrt((x - i)**2 + (y - j)**2)
      gaus = np.exp(-(dst**2)/(2*(sigma**2)))

      diff = np.abs(window[x, y] - window[i,j])
      gaus_diff = np.exp(-(diff**2)/(2*(delta_r**2)))

      pixel = pixel + gaus*gaus_diff*window[i,j]
      normalization +=gaus*gaus_diff
  pixel=pixel/(normalization)
  return pixel

In [ ]:
def bilateral(image, window, sigma, delta_r):
  IConv = image.copy()
  (X,Y) = IConv.shape
  var = window//2
  for i in range(0 + var, X - var):
    for j in range(0 + var, Y - var):
      window = IConv[i-var:i+var+1, j-var:j+var+1]
      new_pixel = new_pixel_value_2(window, sigma, delta_r)
      IConv[i,j] = new_pixel
  return IConv

In [ ]:
size=5
sigma=0.7
delta_r=20
convolucja = convol(Input1, size, sigma)
bilateralne = bilateral(Input1, size, sigma, delta_r)

f, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(16,16))
ax1.imshow(Input1,'gray')
ax1.set_title("Onraz oryginalny")
ax1.axis('off')
ax2.imshow(convolucja, 'gray')
ax2.set_title("Obraz w wyniku konwolucji")
ax2.axis('off')
ax3.imshow(bilateralne, 'gray')
ax3.set_title('Obraz w wyniku filtracji bilateralnej')
ax3.axis('off')

Obraz w wyniku filtracji bilateralnej stał się bardziej wygładzony, białe kropki, które wcześniej występowały nie są już widoczne.

In [ ]:
size=5
sigma=0.7
delta_r=20
convolucja = convol(Input2, size, sigma)
bilateralne = bilateral(Input2, size, sigma, delta_r)

f, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(16,16))
ax1.imshow(Input2,'gray')
ax1.set_title("Onraz oryginalny")
ax1.axis('off')
ax2.imshow(convolucja, 'gray')
ax2.set_title("Obraz w wyniku konwolucji")
ax2.axis('off')
ax3.imshow(bilateralne, 'gray')
ax3.set_title('Obraz w wyniku filtracji bilateralnej')
ax3.axis('off')

In [ ]:
size=5
sigma=0.7
delta_r=20
convolucja = convol(Input3, size, sigma)
bilateralne = bilateral(Input3, size, sigma, delta_r)

f, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(16,16))
ax1.imshow(Input3,'gray')
ax1.set_title("Onraz oryginalny")
ax1.axis('off')
ax2.imshow(convolucja, 'gray')
ax2.set_title("Obraz w wyniku konwolucji")
ax2.axis('off')
ax3.imshow(bilateralne, 'gray')
ax3.set_title('Obraz w wyniku filtracji bilateralnej')
ax3.axis('off')

In [ ]:
size=5
sigma=0.7
delta_r=20
convolucja = convol(Input4, size, sigma)
bilateralne = bilateral(Input4, size, sigma, delta_r)

f, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(16,16))
ax1.imshow(Input4,'gray')
ax1.set_title("Onraz oryginalny")
ax1.axis('off')
ax2.imshow(convolucja, 'gray')
ax2.set_title("Obraz w wyniku konwolucji")
ax2.axis('off')
ax3.imshow(bilateralne, 'gray')
ax3.set_title('Obraz w wyniku filtracji bilateralnej')
ax3.axis('off')

In [ ]:
size=5
sigma=0.7
delta_r=20
convolucja = convol(Input_0, size, sigma)
bilateralne = bilateral(Input_0, size, sigma, delta_r)

f, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(16,16))
ax1.imshow(Input_0,'gray')
ax1.set_title("Onraz oryginalny")
ax1.axis('off')
ax2.imshow(convolucja, 'gray')
ax2.set_title("Obraz w wyniku konwolucji")
ax2.axis('off')
ax3.imshow(bilateralne, 'gray')
ax3.set_title('Obraz w wyniku filtracji bilateralnej')
ax3.axis('off')

Konwlucja powoduje powstanie mniej ostrych krawędzi , natomiast filtracja bilateralna wygładza obrazy i eliminuje niedoskonałości.